In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip install finance-datareader
!pip install neuralprophet[live]

     |████████████████████████████████| 55 kB 2.7 MB/s 
     |████████████████████████████████| 155 kB 17.5 MB/s 
  Attempting uninstall: holidays
    Found existing installation: holidays 0.10.5.2
    Uninstalling holidays-0.10.5.2:
      Successfully uninstalled holidays-0.10.5.2


In [3]:
import numpy as np
import pandas as pd
import gc
import math
import os.path
import time
import matplotlib.pyplot as plt
from datetime import timedelta, datetime
from dateutil import parser
from tqdm import tqdm
import copy
from fbprophet import Prophet

from neuralprophet import NeuralProphet
from neuralprophet import set_random_seed 

import warnings
import datetime
import random
import tensorflow as tf
warnings.filterwarnings("ignore")

# PUBLIC

In [4]:
path = '/content/drive/MyDrive/stock_2/'
list_name = 'stock_list.csv'
sample_name = 'sample_submission.csv'

stock_list = pd.read_csv(os.path.join(path,list_name))
stock_list['종목코드'] = stock_list['종목코드'].apply(lambda x : str(x).zfill(6))
stock_list

,종목명,종목코드,상장시장
0,삼성전자,005930,KOSPI
1,SK하이닉스,000660,KOSPI
2,NAVER,035420,KOSPI
3,카카오,035720,KOSPI
4,삼성바이오로직스,207940,KOSPI
...,...,...,...
365,맘스터치,220630,KOSDAQ
366,다날,064260,KOSDAQ
367,제이시스메디칼,287410,KOSDAQ
368,크리스에프앤씨,110790,KOSDAQ


In [5]:
import os
fileList = os.listdir('/content/drive/MyDrive/stock_2/train')

In [6]:
companyList = []
for file in fileList:
    companyName = file.split("_")[1]
    companyName = companyName.split('.')[0]
    if companyName != "csv":
        companyList.append(companyName)
companyList = sorted(companyList)
print(companyList)

['000060', '000080', '000100', '000120', '000150', '000240', '000250', '000270', '000660', '000670', '000720', '000810', '000880', '000990', '001230', '001440', '001450', '001740', '002380', '002790', '003000', '003090', '003380', '003410', '003490', '003670', '003800', '004000', '004020', '004170', '004370', '004490', '004800', '004990', '005250', '005290', '005300', '005380', '005385', '005387', '005490', '005830', '005850', '005930', '005935', '005940', '006260', '006280', '006360', '006400', '006650', '006730', '006800', '007070', '007310', '007390', '008560', '008770', '008930', '009150', '009240', '009540', '009830', '009900', '010060', '010120', '010130', '010620', '010780', '010950', '011000', '011070', '011170', '011200', '011210', '011780', '012330', '012450', '012510', '012750', '013120', '013890', '014680', '015750', '015760', '016360', '016380', '017670', '017800', '018260', '018880', '019170', '020150', '021240', '022100', '023530', '024110', '025900', '025980', '026960',

In [7]:
sample_submission = pd.read_csv(path + 'sample_submission.csv')
sample_submission = sample_submission[['Day']]
sample_submission['Day'] =pd.to_datetime(sample_submission['Day'])

In [8]:
for i in tqdm(sorted(companyList)):
    globals()['df_{}'.format(i)] = pd.read_csv('/content/drive/MyDrive/stock_2/train/stock_' + i + ".csv")
    locals()['df_{}'.format(i)] = locals()['df_{}'.format(i)][['Date', 'Close']]
    locals()['df_{}'.format(i)].rename(columns={'Date':'ds'}, inplace=True)
    locals()['df_{}'.format(i)].rename(columns={'Close':'y'}, inplace=True)

100%|██████████| 370/370 [01:46<00:00,  3.47it/s]


In [9]:
# model = NeuralProphet(
#     n_forecasts=5,
#         n_lags=370,
#         yearly_seasonality=False,
#         weekly_seasonality=False,
#         daily_seasonality=False,
#         batch_size=64,
#         epochs=100,
#         learning_rate=1.0,
# )

# metric = model.fit(df_000060, freq='D', valid_p=0.1, epochs=100)

# future = model.make_future_dataframe(df_000060, periods=7)
# forecast = model.predict(future)

In [ ]:
for i in sorted(companyList):
    globals()['model_{}'.format(i)] = NeuralProphet(
        n_forecasts=5,
        n_lags=20,
        yearly_seasonality='auto',
        weekly_seasonality='auto',
        daily_seasonality='auto',
        batch_size=64,
        epochs=100,
        learning_rate=0.1,
        loss_func='mae',
        normalize='auto'
        )

    globals()['metric_{}'.format(i)] = locals()['model_{}'.format(i)].fit(locals()['df_{}'.format(i)],
    freq='D',
    valid_p=0.1,
    epochs=100)

    globals()['future_{}'.format(i)] = locals()['model_{}'.format(i)].make_future_dataframe(locals()['df_{}'.format(i)], periods=5)
    globals()['forecast_{}'.format(i)] = locals()['model_{}'.format(i)].predict(locals()['future_{}'.format(i)])

In [12]:
forecast_000060

,ds,y,yhat1,residual1,yhat2,residual2,yhat3,residual3,yhat4,residual4,yhat5,residual5,ar1,ar2,ar3,ar4,ar5,trend,season_weekly
0,2021-09-30,30550.000000,None,NaN,None,NaN,None,NaN,None,NaN,None,NaN,None,None,None,None,None,None,None
1,2021-10-01,30300.000000,None,NaN,None,NaN,None,NaN,None,NaN,None,NaN,None,None,None,None,None,None,None
2,2021-10-02,30287.500000,None,NaN,None,NaN,None,NaN,None,NaN,None,NaN,None,None,None,None,None,None,None
3,2021-10-03,30275.000000,None,NaN,None,NaN,None,NaN,None,NaN,None,NaN,None,None,None,None,None,None,None
4,2021-10-04,30262.500000,None,NaN,None,NaN,None,NaN,None,NaN,None,NaN,None,None,None,None,None,None,None
5,2021-10-05,30250.000000,None,NaN,None,NaN,None,NaN,None,NaN,None,NaN,None,None,None,None,None,None,None
6,2021-10-06,29750.000000,None,NaN,None,NaN,None,NaN,None,NaN,None,NaN,None,None,None,None,None,None,None
7,2021-10-07,28900.000000,None,NaN,None,NaN,None,NaN,None,NaN,None,NaN,None,None,None,None,None,None,None
8,2021-10-08,29250.000000,None,NaN,None,NaN,None,NaN,None,NaN,None,NaN,None,None,None,None,None,None,None
9,2021-10-09,29287.500000,None,NaN,None,NaN,None,NaN,None,NaN,None,NaN,None,None,None,None,None,None,None


In [13]:
for i in sorted(companyList):
    globals()['list_{}'.format(i)] = []
    locals()['list_{}'.format(i)].append(locals()['forecast_{}'.format(i)]['yhat1'][30])
    locals()['list_{}'.format(i)].append(locals()['forecast_{}'.format(i)]['yhat2'][31])
    locals()['list_{}'.format(i)].append(locals()['forecast_{}'.format(i)]['yhat3'][32])
    locals()['list_{}'.format(i)].append(locals()['forecast_{}'.format(i)]['yhat4'][33])
    locals()['list_{}'.format(i)].append(locals()['forecast_{}'.format(i)]['yhat5'][34])

In [14]:
sample_submission = pd.read_csv(path + 'sample_submission.csv')
sub_public = sample_submission[0:5]
sub_private = sample_submission[5:10]

In [15]:
for i in sorted(companyList):
    sub_public[i] = locals()['list_{}'.format(i)]

In [17]:
import os
fileList = os.listdir('/content/drive/MyDrive/stock_2/test')

In [18]:
companyList = []
for file in fileList:
    companyName = file.split("_")[1]
    companyName = companyName.split('.')[0]
    if companyName != "csv":
        companyList.append(companyName)
companyList = sorted(companyList)
print(companyList)

['000060', '000080', '000100', '000120', '000150', '000240', '000250', '000270', '000660', '000670', '000720', '000810', '000880', '000990', '001230', '001440', '001450', '001740', '002380', '002790', '003000', '003090', '003380', '003410', '003490', '003670', '003800', '004000', '004020', '004170', '004370', '004490', '004800', '004990', '005250', '005290', '005300', '005380', '005385', '005387', '005490', '005830', '005850', '005930', '005935', '005940', '006260', '006280', '006360', '006400', '006650', '006730', '006800', '007070', '007310', '007390', '008560', '008770', '008930', '009150', '009240', '009540', '009830', '009900', '010060', '010120', '010130', '010620', '010780', '010950', '011000', '011070', '011170', '011200', '011210', '011780', '012330', '012450', '012510', '012750', '013120', '013890', '014680', '015750', '015760', '016360', '016380', '017670', '017800', '018260', '018880', '019170', '020150', '021240', '022100', '023530', '024110', '025900', '025980', '026960',

In [19]:
for i in tqdm(sorted(companyList)):
    globals()['df_{}'.format(i)] = pd.read_csv('/content/drive/MyDrive/stock_2/test/stock_' + i + ".csv")
    locals()['df_{}'.format(i)] = locals()['df_{}'.format(i)][['Date', 'Close']]
    locals()['df_{}'.format(i)].rename(columns={'Date':'ds'}, inplace=True)
    locals()['df_{}'.format(i)].rename(columns={'Close':'y'}, inplace=True)

100%|██████████| 370/370 [01:51<00:00,  3.32it/s]


In [20]:
for i in sorted(companyList):
    globals()['model_{}'.format(i)] = NeuralProphet(
        n_forecasts=5,
        n_lags=20,
        yearly_seasonality='auto',
        weekly_seasonality='auto',
        daily_seasonality='auto',
        batch_size=64,
        epochs=100,
        learning_rate=0.1,
        loss_func='mae',
        normalize='auto'
        )

    globals()['metric_{}'.format(i)] = locals()['model_{}'.format(i)].fit(locals()['df_{}'.format(i)],
    freq='D',
    valid_p=0.1,
    epochs=100)

    globals()['future_{}'.format(i)] = locals()['model_{}'.format(i)].make_future_dataframe(locals()['df_{}'.format(i)], periods=5)
    globals()['forecast_{}'.format(i)] = locals()['model_{}'.format(i)].predict(locals()['future_{}'.format(i)])

Epoch[100/100]: 100%|██████████| 100/100 [00:03<00:00, 26.86it/s, L1Loss=0.0619, MAE=3.16e+3, MSE=1.93e+7, RegLoss=0]
INFO - (NP.forecaster._handle_missing_data) - 6 missing dates added.
INFO - (NP.forecaster._handle_missing_data) - 6 NaN values in column y were auto-imputed.
INFO - (NP.forecaster._handle_missing_data) - 103 missing dates added.
INFO - (NP.forecaster._handle_missing_data) - 103 NaN values in column y were auto-imputed.
INFO - (NP.utils.set_auto_seasonalities) - Disabling yearly seasonality. Run NeuralProphet with yearly_seasonality=True to override this.
INFO - (NP.utils.set_auto_seasonalities) - Disabling daily seasonality. Run NeuralProphet with daily_seasonality=True to override this.
Epoch[100/100]: 100%|██████████| 100/100 [00:03<00:00, 30.14it/s, L1Loss=0.0543, MAE=1.19e+3, MSE=2.9e+6, RegLoss=0]
INFO - (NP.forecaster._handle_missing_data) - 6 missing dates added.
INFO - (NP.forecaster._handle_missing_data) - 6 NaN values in column y were auto-imputed.
INFO - (NP

In [22]:
for i in sorted(companyList):
    globals()['list_{}'.format(i)] = []
    locals()['list_{}'.format(i)].append(locals()['forecast_{}'.format(i)]['yhat1'][26])
    locals()['list_{}'.format(i)].append(locals()['forecast_{}'.format(i)]['yhat2'][27])
    locals()['list_{}'.format(i)].append(locals()['forecast_{}'.format(i)]['yhat3'][28])
    locals()['list_{}'.format(i)].append(locals()['forecast_{}'.format(i)]['yhat4'][29])
    locals()['list_{}'.format(i)].append(locals()['forecast_{}'.format(i)]['yhat5'][30])

In [23]:
for i in sorted(companyList):
    sub_private[i] = locals()['list_{}'.format(i)]

In [24]:
sub_total = pd.concat([sub_public, sub_private])

In [25]:
sub_total.to_csv(path + 'neural_prophet_lag_20.csv', index=False)